In [1]:
from alco_analysis.loading_data import load_data
from alco_analysis.correlation import *
from alco_analysis.stats import count_concession, summary, merge_df_on_city, different_correlations

df_conc, df_events, df_cities = load_data("data/concession.csv", 
                                          "data/events.csv",
                                          "data/cities.csv")


###CONCESSION DATAFRAME###

Deleted 0 rows

###EVENTS DATAFRAME###

Deleted 0 rows

###CITIES DATAFRAME###

Deleted 0 rows


In [2]:
merged = (df_conc.merge(df_cities, on=['city'], how='left', indicator=True)) 
print("MERGED CONCESSION DATA FRAME AND CITIES -> LOCALIZATION OF EVERY CONCESSION")
# print(merged.head())

vc = merged['_merge'].value_counts()

print("Dopasowane   :", vc.get('both', 0))
print("Niedopasowane:", vc.get('left_only', 0))
print("Nie­użyte     :", vc.get('right_only', 0))

missing = merged.loc[merged['_merge'] == 'left_only', 'city']

print("Koncesje bez lokalizacji przydzielonego miasta")
print(missing.unique())
percent = vc.get('left_only', 0)/(vc.get('both', 0) + vc.get('left_only', 0))*100

print(f"\n### Dropping {percent:.2f}% of data ###")
merged.dropna(subset=["lon", "lat"], inplace=True)

MERGED CONCESSION DATA FRAME AND CITIES -> LOCALIZATION OF EVERY CONCESSION
Dopasowane   : 394
Niedopasowane: 33
Nie­użyte     : 0
Koncesje bez lokalizacji przydzielonego miasta
['bienkowka' 'nowy  tomysl' 'bielsko - biala' 'libertow' 'dobron'
 'banska nizna' 'tenczynek' 'stare stracze' 'modlniczka' 'tomaszkowo'
 'dziekanow nowy' 'poronin' 'wegrzce' 'uciechow' 'kokoszkowy' 'siedlecin'
 'weglowice' 'proszowki' 'zascianki' 'srock' 'chelm slaski' 'dys'
 'busko-zdroj' 'plock-biala' 'stary bartkow' 'sekocin nowy' 'miedzychow'
 'zbylitowska gora' 'nowiny' 'gilowice' 'stare proboszczewice' 'wolkowo']

### Dropping 7.73% of data ###


# Obliczanie korelacji między pożarami a merged (koncesje & lokalizacja)

In [ ]:
df_conc_count = count_concession(merged)

df_corr = (df_conc_count.merge(df_events, on='city', how='inner'))


print(df_conc_count)
pearson = df_corr[['Liczba_koncesji', 'RAZEM Pożar (P)']].corr().iloc[0,1]
print("Pearson =", round(pearson, 3))